In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# set initial 

In [ ]:
# time stepper
class Cell:
    '''
    '''

    def __init__(self,name,dconc,iconc,nconc,urea):
        # Create a cell instance and rate constants
        self.name = name
        self.dconc = dconc
        self.iconc = iconc
        self.nconc = nconc
        
        self.ku_15 = 
        self.ku_16
        self.kf_15
        self.kf_16

    
    




In [ ]:
class system:
    '''

    '''

    def __init__(self,conc_0,)